In [ ]:
import pandas as pd
import torch
import pickle
import matplotlib.pyplot as plt
data = pickle.load(open('/content/drive/MyDrive/docs.pkl',"rb"))

In [ ]:
documents = data['train']
documents[0]['category']

'technologie'

In [ ]:
# 12번 문서의 원문과 단어 등장 횟수 데이터 추출
document_info = data['train'][12]

# 단어 등장 횟수 데이터 (사전 형식) 추출
word_count = document_info['words']

# 가장 많이 등장한 단어 5개 찾기
top_words = sorted(word_count.items(), key=lambda x: x[1], reverse=True)[:5]

# 결과 출력
print("가장 많이 등장한 단어 5개:")
for word, count in top_words:
    print(f"{word}: {count}")

# document_info

가장 많이 등장한 단어 5개:
the: 22
in: 14
to: 13
a: 12
conte: 11


In [ ]:
from sklearn.metrics import jaccard_score
import numpy as np

# 31번 문서와 다른 모든 문서의 Jaccard 유사도 계산
jaccard_scores = []
document_31 = set(data['train'][31]['words'])

for i, document in enumerate(data['train']):
    if i != 31:  # 31번 문서와 자기 자신은 비교하지 않음
        document_i = set(document['words'])
        jaccard_similarity = len(document_31.intersection(document_i)) / len(document_31.union(document_i))
        jaccard_scores.append((i, jaccard_similarity))

# Jaccard 유사도가 높은 상위 5개 문서 찾기
top_similar_documents = sorted(jaccard_scores, key=lambda x: x[1], reverse=True)[:5]

# 결과 출력
print(f"31번 문서와 가장 유사한 문서 5개:")
for i, similarity in top_similar_documents:
    print(f"문서 {i}: Jaccard 유사도 = {similarity:.2f}")


31번 문서와 가장 유사한 문서 5개:
문서 182: Jaccard 유사도 = 1.00
문서 323: Jaccard 유사도 = 0.20
문서 659: Jaccard 유사도 = 0.18
문서 749: Jaccard 유사도 = 0.18
문서 742: Jaccard 유사도 = 0.18


In [ ]:
data['train'][12]

In [ ]:
# 가장 유사한 문서들의 인덱스
similar_document_indices = [i for i, _ in top_similar_documents]

# 각 유사한 문서의 카테고리 가져오기
similar_document_categories = [documents[i]['category'] for i in similar_document_indices]

# 결과 출력
print(f"유사한 5개 문서의 카테고리:")
for i, category in enumerate(similar_document_categories):
    print(f"유사한 문서 {i + 1}의 카테고리: {category}")


유사한 5개 문서의 카테고리:
유사한 문서 1의 카테고리: technologie
유사한 문서 2의 카테고리: technologie
유사한 문서 3의 카테고리: technologie
유사한 문서 4의 카테고리: technologie
유사한 문서 5의 카테고리: politics


In [ ]:
print(documents[182]['category'],
documents[323]['category'],
documents[659]['category'],
documents[749]['category'],
documents[742]['category'])

technologie technologie technologie technologie politics


In [ ]:
from collections import defaultdict
from sklearn.metrics import jaccard_score

# 각 단어와 그 단어가 포함된 문서들의 집합을 저장하는 사전 생성
word_document_sets = defaultdict(set)

# 모든 단어와 해당 단어를 포함하는 문서 인덱스 집합 생성
for i, document in enumerate(data['train']):
    words = document['words']
    for word in words:
        word_document_sets[word].add(i)

# "game"과 다른 단어들 간의 Jaccard 유사도 계산
target_word = "game"
similarities = {}
for word in word_document_sets.keys():
    if word != target_word:
        intersection = len(word_document_sets[target_word].intersection(word_document_sets[word]))
        union = len(word_document_sets[target_word].union(word_document_sets[word]))
        jaccard_similarity = intersection / union
        similarities[word] = jaccard_similarity

# Jaccard 유사도가 가장 높은 단어 5개 찾기
top_similar_words = sorted(similarities.items(), key=lambda x: x[1], reverse=True)[:20]

# 결과 출력
print(f"'game'과 Jaccard 유사도가 가장 높은 단어 5개:")
for word, similarity in top_similar_words:
    print(f"{word}: Jaccard 유사도 = {similarity:.5f}")


'game'과 Jaccard 유사도가 가장 높은 단어 5개:
gaming: Jaccard 유사도 = 0.20000
nintendo: Jaccard 유사도 = 0.18182
sony: Jaccard 유사도 = 0.17857
video: Jaccard 유사도 = 0.17647
sonys: Jaccard 유사도 = 0.17391
games: Jaccard 유사도 = 0.16667
ball: Jaccard 유사도 = 0.16000
runners: Jaccard 유사도 = 0.14286
perspective: Jaccard 유사도 = 0.13793
features: Jaccard 유사도 = 0.13636
preserve: Jaccard 유사도 = 0.13636
consoles: Jaccard 유사도 = 0.13636
titles: Jaccard 유사도 = 0.13333
option: Jaccard 유사도 = 0.12903
kit: Jaccard 유사도 = 0.12500
includes: Jaccard 유사도 = 0.11628
tasks: Jaccard 유사도 = 0.11538
corner: Jaccard 유사도 = 0.11538
playing: Jaccard 유사도 = 0.11429
preparation: Jaccard 유사도 = 0.11111


In [ ]:
import math

# 31번 문서 추출
document_31 = data['train'][31]

# TFIDF 점수를 저장할 딕셔너리 생성
tfidf_scores = {}

# IDF 계산
total_documents = len(data['train'])
for word, tf in document_31['words'].items():
    # 단어 t를 포함하는 문서 수 계산
    documents_containing_word = sum(1 for document in data['train'] if word in document['words'])

    # IDF 계산
    idf = math.log(total_documents / (documents_containing_word + 1))  # +1은 분모가 0이 되지 않도록 하기 위함

    # TFIDF 계산
    tfidf = tf * idf
    tfidf_scores[word] = tfidf

# TFIDF 점수가 가장 높은 단어 5개 찾기
top_tfidf_words = sorted(tfidf_scores.items(), key=lambda x: x[1], reverse=True)[:5]

# 결과 출력
print(f"31번 문서에서 TFIDF 점수가 가장 높은 단어 5개:")
for word, tfidf in top_tfidf_words:
    print(f"{word}: TFIDF = {tfidf:.4f}")


31번 문서에서 TFIDF 점수가 가장 높은 단어 5개:
laptops: TFIDF = 31.1577
negroponte: TFIDF = 28.5189
laptop: TFIDF = 25.9648
child: TFIDF = 16.9498
digital: TFIDF = 11.5615


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 31번 문서의 임베딩 벡터 추출
document_31_embedding = np.array(data['train'][31]['embedding']).reshape(1, -1)

# 다른 모든 문서의 임베딩 벡터 추출 및 인덱스 저장
similar_documents_with_indices = []

for i, document in enumerate(data['train']):
    if i != 31:  # 31번 문서와 자기 자신은 비교하지 않음
        document_i_embedding = np.array(document['embedding']).reshape(1, -1)
        similarity_score = cosine_similarity(document_31_embedding, document_i_embedding)[0][0]
        similar_documents_with_indices.append((i, similarity_score))

# 유사도가 높은 상위 5개 문서 찾기
top_similar_documents = sorted(similar_documents_with_indices, key=lambda x: x[1], reverse=True)[:5]

# 결과 출력
print(f"31번 문서와 가장 유사한 문서 5개:")
for i, similarity in top_similar_documents:
    print(f"문서 {i}: Cosine 유사도 = {similarity:.4f}")


31번 문서와 가장 유사한 문서 5개:
문서 182: Cosine 유사도 = 0.9998
문서 749: Cosine 유사도 = 0.7467
문서 72: Cosine 유사도 = 0.6970
문서 66: Cosine 유사도 = 0.6534
문서 302: Cosine 유사도 = 0.6500


In [ ]:
# 가장 유사한 문서들의 인덱스
similar_document_indices = [i for i, _ in top_similar_documents]

# 각 유사한 문서의 카테고리 가져오기
similar_document_categories = [documents[i]['category'] for i in similar_document_indices]

# 결과 출력
print(f"유사한 5개 문서의 카테고리:")
for i, similarity in top_similar_documents:
    print(f"문서 {i}: 의 카테고리: {category}")

print(data['train'][31]['category'])

유사한 5개 문서의 카테고리:
문서 182: 의 카테고리: technologie
문서 323: 의 카테고리: technologie
문서 659: 의 카테고리: technologie
문서 749: 의 카테고리: technologie
문서 742: 의 카테고리: technologie
technologie


In [ ]:
from sklearn.metrics import jaccard_score
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from scipy.stats import pearsonr

# 31번 문서의 단어 집합 추출
document_31_words = set(data['train'][31]['words'])

# Jaccard 유사도와 Cosine 유사도를 저장할 리스트 초기화
jaccard_similarities = []
cosine_similarities = []

for i, document in enumerate(data['train']):
    if i != 31:  # 31번 문서와 자기 자신은 비교하지 않음
        # 다른 문서의 단어 집합 추출
        document_i_words = set(document['words'])

        # Jaccard 유사도 계산
        jaccard_similarity = len(document_31_words.intersection(document_i_words)) / len(document_31_words.union(document_i_words))
        jaccard_similarities.append(jaccard_similarity)

        # Cosine 유사도 계산
        document_31_embedding = np.array(data['train'][31]['embedding']).reshape(1, -1)
        document_i_embedding = np.array(document['embedding']).reshape(1, -1)
        cosine_similarity_value = cosine_similarity(document_31_embedding, document_i_embedding)[0][0]
        cosine_similarities.append(cosine_similarity_value)

# Jaccard 유사도와 Cosine 유사도 사이의 상관 계수 계산
correlation_coefficient = pearsonr(jaccard_similarities, cosine_similarities)

# 결과 출력
print(f"Jaccard 유사도와 Cosine 유사도의 상관 계수: {correlation_coefficient[0]:.8f}")


Jaccard 유사도와 Cosine 유사도의 상관 계수: 0.55463512


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# 데이터 준비
X_train = [document['embedding'] for document in data['train']]
y_train = [document['category'] for document in data['train']]
X_test = [document['embedding'] for document in data['test']]

# 표준화 (Standardization) - 필요에 따라 스케일링을 수행할 수 있습니다.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 모델 초기화
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')

# 모델 학습
model.fit(X_train, y_train)

# 테스트 데이터 예측
predicted_labels = model.predict(X_test)

# 예측 결과 출력
for i, label in enumerate(predicted_labels):
    print(f"문서 {i}의 예측된 레이블: {label}")


문서 0의 예측된 레이블: food
문서 1의 예측된 레이블: business
문서 2의 예측된 레이블: historical
문서 3의 예측된 레이블: medical
문서 4의 예측된 레이블: entertainment
문서 5의 예측된 레이블: sport
문서 6의 예측된 레이블: entertainment
문서 7의 예측된 레이블: historical
문서 8의 예측된 레이블: technologie
문서 9의 예측된 레이블: politics
